### Import Libaries

In [1]:
from serpapi import GoogleSearch
from datetime import date,timedelta,datetime
import numpy as np
import pandas as pd
import time
import sys
pd.set_option('display.max_columns', None)

### Load Dataset

In [29]:
airports = pd.read_csv('major_international_airports_clean.csv')
temp = airports[airports['IATA_Code'].isin(['DUB','AKL'])]
airports = airports[:len(airports)-20] # take N  rows
airports = pd.concat([airports,temp], ignore_index=True).drop_duplicates().reset_index(drop=True)
display(airports)

,IATA_Code,Airport_Name,City,Country
0,DUB,Dublin Airport,Dublin,Ireland
1,AKL,Auckland Airport,Auckland,New Zealand
2,YYZ,Toronto Pearson International Airport,Toronto,Canada
3,YVR,Vancouver International Airport,Vancouver,Canada
4,YUL,Montréal-Trudeau International Airport,Montreal,Canada
5,YEG,Edmonto International Airport,Edmonto,Canada
6,YYC,Calgary International Airport,Calgary,Canada
7,JFK,John F. Kennedy International Airport,New York,USA
8,LGA,LaGuardia Airport,New York,USA
9,EWR,Newark Liberty International Airport,Newark,USA


### Fight Data Extractor

In [30]:
flghDta = pd.DataFrame(columns=[
        'apt_name_dp','apt_IATA_dp','apt_time_dt_dp','apt_time_tm_dp',
        'apt_name_ds','apt_IATA_ds','apt_time_dt_ds','apt_time_tm_ds',
        'trip_class','price','ticket_sold_by','travel_type'])
def dataFightExt(data):
    for fd in data:
        fligths = fd['flights'][0]
      
        dp,ds= fligths['departure_airport'],fligths['arrival_airport']
        dp_name,ds_name = dp['name'],ds['name']
        dp_IATA,ds_IATA = dp['id'],ds['id'] 

        dp_dttm,ds_dttm = dp['time'],ds['time']
        dp_dttm_obj = datetime.strptime(dp_dttm,'%Y-%m-%d %H:%M')
        ds_dttm_obj = datetime.strptime(ds_dttm,'%Y-%m-%d %H:%M')
        dp_dt,dp_tm = dp_dttm_obj.date(),dp_dttm_obj.time()
        ds_dt,ds_tm = ds_dttm_obj.date(),ds_dttm_obj.time()

        trip_class = fligths['travel_class']
        try:
            tck_sld_by = fligths['ticket_also_sold_by']
        except:
            tck_sld_by = []

        price = fd['price']
        travel_type = fd['type']
        flghDta.loc[len(flghDta)] = [
            dp_name,dp_IATA,dp_dt,dp_tm,
            ds_name,ds_IATA,ds_dt,ds_tm,
            trip_class,price,tck_sld_by,travel_type]

### SerpAPI for Future Flight Data

In [ ]:
i = -1 # When issuess happen set this to the last value/completed here to resume after fixing said issuess with code to mange unexpected errors
outbound = date(2025,11,29)
EndPoint = date(2026,2,15)
des = ['DUB','AKL']

Limit = False
for idx,d in airports.iterrows():
  dep = d['IATA_Code'] 
  if idx < i: continue

  hold = ''
  if dep in des: hold = des.pop(des.index(dep))
  
  while outbound <= EndPoint:
    print(f"{idx} || {outbound}//{EndPoint} || DEP: {dep} || DES: {des}", end='\r')

    params = {
      "api_key": "API_KEY",
      "engine": "google_flights",
      "hl": "en",
      "gl": "ca",
      "departure_id": dep,
      "arrival_id": ','.join(des),
      "outbound_date": str(outbound),
      "currency": "CAD",
      "sort_by": "3",
      "type": "2",
      "stops": "1"
    }

    chkact = GoogleSearch(params).get_account()
    if chkact['this_month_usage'] > 4890:
      Limit = True
      break
  
    search = GoogleSearch(params)
    results = search.get_dict()

    try:
      dataFightExt(results['other_flights'])
    except Exception as e:
      print(e)

    outbound += timedelta(days=7)
    
    sys.stdout.flush()
    time.sleep(0.1)

  # print(f"")

  if Limit == True: break

  if hold != '': des.append(dep)
  
  outbound = date(2025,11,29)
  
flghDta = flghDta.convert_dtypes()
display(flghDta.head(5))

'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: DUB || DES: ['AKL']
'other_flights'//2026-02-15 || DEP: AKL || DES: ['DUB']
'other_flights'//2026-02-15 || DEP: AKL || DES: ['DUB']
'other_flights'//2026-02-15 || DEP: AKL || DES: ['DUB']
'other_flights'//2026-02-15 || DEP: AKL || DES: ['DUB']
'other_flights'//2026-02-15 || DEP: AKL || DES: ['DUB']
'other_flights'//2026-02-15 || DEP: AKL || DES: 

,apt_name_dp,apt_IATA_dp,apt_time_dt_dp,apt_time_tm_dp,apt_name_ds,apt_IATA_ds,apt_time_dt_ds,apt_time_tm_ds,trip_class,price,ticket_sold_by,travel_type
0,Toronto Pearson International Airport,YYZ,2025-11-29,17:25:00,Dublin Airport,DUB,2025-11-30,05:15:00,Economy,766,[American],One way
1,Toronto Pearson International Airport,YYZ,2025-11-29,19:35:00,Dublin Airport,DUB,2025-11-30,07:10:00,Economy,580,[Lufthansa],One way
2,Toronto Pearson International Airport,YYZ,2025-12-06,17:25:00,Dublin Airport,DUB,2025-12-07,05:15:00,Economy,766,[American],One way
3,Toronto Pearson International Airport,YYZ,2025-12-06,19:35:00,Dublin Airport,DUB,2025-12-07,07:10:00,Economy,1097,[Lufthansa],One way
4,Toronto Pearson International Airport,YYZ,2025-12-13,17:25:00,Dublin Airport,DUB,2025-12-14,05:15:00,Economy,959,[American],One way


In [32]:
flghDta.insert(4,'City_dp',None)
flghDta.insert(4,'Country_dp',None)
flghDta.insert(10,'City_ds',None)
flghDta.insert(11,'Country_ds',None)
flghDta.head(3)

,apt_name_dp,apt_IATA_dp,apt_time_dt_dp,apt_time_tm_dp,Country_dp,City_dp,apt_name_ds,apt_IATA_ds,apt_time_dt_ds,apt_time_tm_ds,City_ds,Country_ds,trip_class,price,ticket_sold_by,travel_type
0,Toronto Pearson International Airport,YYZ,2025-11-29,17:25:00,None,None,Dublin Airport,DUB,2025-11-30,05:15:00,None,None,Economy,766,[American],One way
1,Toronto Pearson International Airport,YYZ,2025-11-29,19:35:00,None,None,Dublin Airport,DUB,2025-11-30,07:10:00,None,None,Economy,580,[Lufthansa],One way
2,Toronto Pearson International Airport,YYZ,2025-12-06,17:25:00,None,None,Dublin Airport,DUB,2025-12-07,05:15:00,None,None,Economy,766,[American],One way


In [33]:
flghDtacpy = flghDta.copy()
for idx,d in airports.iterrows():
    code = d['IATA_Code']
    Country = d['Country']
    City = d['City']
    flghDtacpy.loc[flghDta['apt_IATA_dp'] == code,'City_dp'] = City
    flghDtacpy.loc[flghDta['apt_IATA_dp'] == code,'Country_dp'] = Country
    flghDtacpy.loc[flghDta['apt_IATA_ds'] == code,'City_ds'] = City
    flghDtacpy.loc[flghDta['apt_IATA_ds'] == code,'Country_ds'] = Country
flghDtacpy.isnull().any()

apt_name_dp       False
apt_IATA_dp       False
apt_time_dt_dp    False
apt_time_tm_dp    False
Country_dp        False
City_dp           False
apt_name_ds       False
apt_IATA_ds       False
apt_time_dt_ds    False
apt_time_tm_ds    False
City_ds           False
Country_ds        False
trip_class        False
price             False
ticket_sold_by    False
travel_type       False
dtype: bool

In [36]:
output_path = "flight_paths_nc.csv"
flghDtacpy.to_csv(output_path, index=False)

In [37]:
flghDtacpy.shape

(992, 16)